In [29]:
import pandas as pd
data = pd.read_excel(r'C:\Users\cxy\OneDrive\桌面\回国\照明测试.xlsx',sheet_name='Sheet2')#,sheet_name='1001-3000'
file_path = r'C:\Users\cxy\OneDrive\桌面\回国\照明测试新.xlsx'
df = data.reset_index().drop(columns='index')
df['医院名称'] = df['医院名称'].astype('string')
df

,医院名称
0,雷士国际
1,同方友友
2,壹照明
3,蚬壳电业
4,天宝集团
...,...
96,汉光电气
97,电科电源
98,三星照明
99,齐普光电


In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
import time

def setup_driver():
    path_to_chromedriver = r"D:\chromedriver-win64\chromedriver-win64\chromedriver.exe"
    service = Service(executable_path=path_to_chromedriver)
    options = webdriver.ChromeOptions()
    options.headless = True
    driver = webdriver.Chrome(service=service, options=options)
    return driver

def fetch_company_info(query_name, driver):
    url = f'https://aiqicha.baidu.com/s?q={query_name}'
    driver.get(url)
    time.sleep(5)  # 初始访问可能需要更长时间来处理加载
    wait = WebDriverWait(driver, 15)
    try:
        results = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.title a')))
    except TimeoutException:
        print("Timed out waiting for page to load")
        #break
        return query_name, '页面加载超时', '无有效链接'
    #print(results.text)
    found = False
    keywords = ['A股', '新三板定增', '新三板','股权转让','战略投资','被收购']
    for result in results[:7]:
        try:
            tags_list_element = result.find_element(By.XPATH, "./ancestor::div[contains(@class,'items')]//div[@class='tags-list']")
            tags = tags_list_element.find_elements(By.CSS_SELECTOR, '.zx-ent-tag')
            #for tag in tags:
            #    print(tag.text)
            #print("1")    
            #if any('A股' in tag.text for tag in tags):
            if any(any(keyword in tag.text for keyword in keywords) for tag in tags):
            #新三板定增，新三板，股权转让，战略投资，被收购
                company_name = result.get_attribute('title')  # 获取公司名称
                data_log_title = result.get_attribute('data-log-title')  # 获取data-log-title属性
                company_id = data_log_title.split('-')[-1]
                detail_url = f'https://aiqicha.baidu.com/company_detail_{company_id}'
                found = True
                break
        except NoSuchElementException:
            #print("Tags list not found for result:", result.text)
            continue  # 如果找不到tags-list，继续检查下一个结果

        except Exception as e:
        #print(f"Error processing result {result.text}: {str(e)}")
            continue
    
    if not found:
        return query_name, '未找到符合条件的公司', '无有效链接'
    return query_name, company_name, detail_url

driver = setup_driver()
#query_name = "芯瑞达"
#query_name,company_name, detail_url = fetch_company_info(query_name, driver)
results = [fetch_company_info(name, driver) for name in df['医院名称']]
driver.quit()
results_df = pd.DataFrame(results, columns=['查询名称', '类似名称', 'URL'])
results_df
#print(f"公司名称: {company_name}")
#print(f"详细链接: {detail_url}")


,查询名称,类似名称,URL
0,雷士国际,未找到符合条件的公司,无有效链接
1,同方友友,未找到符合条件的公司,无有效链接
2,壹照明,未找到符合条件的公司,无有效链接
3,蚬壳电业,未找到符合条件的公司,无有效链接
4,天宝集团,承德天宝矿业集团有限公司,https://aiqicha.baidu.com/company_detail_29020...
...,...,...,...
96,汉光电气,广州汉光电气股份有限公司,https://aiqicha.baidu.com/company_detail_10769...
97,电科电源,深圳市电科电源股份有限公司,https://aiqicha.baidu.com/company_detail_30813...
98,三星照明,济南三星照明科技股份有限公司,https://aiqicha.baidu.com/company_detail_30003...
99,齐普光电,深圳市齐普光电子股份有限公司,https://aiqicha.baidu.com/company_detail_83561...


In [27]:
results_df.to_excel(file_path)